# Differential Privacy using Tensorflow Privacy (TF Privacy)
TF Privacy is part of Tensorflow's supprot for Responsible AI Toolkit 
In this example we will the SGDOptimizer to demonstrate how to introduce Differential Privacy in Model Training. 
You can find more information on TF Secure at  https://www.tensorflow.org/responsible_ai/privacy/guide

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

2024-06-29 22:00:11.651312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 22:00:11.651341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 22:00:11.652380: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load the CIFAR-10 dataset and build a simple CNN model

In [6]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

## Train the model with the DPKerasSGDOptimizer

In [7]:
# Differential Privacy parameters
noise_multiplier = 0.5  # Reduced noise multiplier
l2_norm_clip = 1.5      # Increased gradient clipping
batch_size = 256        # Increased batch size
learning_rate = 0.01
num_microbatches = 1    # Set a reasonable number of microbatches


# Compile the model using DPKerasSGDOptimizer
optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/10
196/196 [==============================] - 3s 7ms/step - loss: 2.2847 - accuracy: 0.1402 - val_loss: 2.2476 - val_accuracy: 0.1799
Epoch 2/10
196/196 [==============================] - 1s 4ms/step - loss: 2.2434 - accuracy: 0.1847 - val_loss: 2.3227 - val_accuracy: 0.1660
Epoch 3/10
196/196 [==============================] - 1s 4ms/step - loss: 2.3173 - accuracy: 0.1786 - val_loss: 2.3223 - val_accuracy: 0.1880
Epoch 4/10
196/196 [==============================] - 1s 4ms/step - loss: 2.4117 - accuracy: 0.1887 - val_loss: 2.5058 - val_accuracy: 0.2066
Epoch 5/10
196/196 [==============================] - 1s 5ms/step - loss: 2.5590 - accuracy: 0.1725 - val_loss: 2.6081 - val_accuracy: 0.1685
Epoch 6/10
196/196 [==============================] - 1s 4ms/step - loss: 2.5008 - accuracy: 0.1806 - val_loss: 2.4151 - val_accuracy: 0.1795
Epoch 7/10
196/196 [==============================] - 1s 4ms/step - loss: 2.3837 - accuracy: 0.1850 - val_loss: 2.3656 - val_accuracy: 0.1799
Epoch 

You will notice the new model has low accuracy.  That is expected to a degree because differential privacy ads noise. However, in our example the model is too simple and that is the main reason of the low accuracy/
The notebook is to show you the technique. You will need to experiment with parameters and architecture and that will be done by an AI Engineer or Data Scientist